In [85]:
import re
import os, sys
import string
import pandas as pd 
import spacy 
import numpy as np
from nltk import *
import operator
df_file = pd.read_csv('FeedbackLanguage_wTags_Interns.csv')

df_cp = pd.DataFrame(df_file)
df = pd.DataFrame(df_file)

df[df.CR_response == 1].message
responses = df[df.CR_response == 1].message
responses

11       Great! Then I will create a request to put your account into ''retry'' state.;↵Thank you,AnnTC S...
20       You can add your father's bank account. Many property;managers are adding owners accounts to sen...
22       Hello, Rudy! Please excuse for the response delay due to Season Holidays.If you want to view all...
23       Hello, Lifu;Wang!Please, review the attached PDFs with the directions.;If you need any further i...
24       Hello there. Currently, there is no functionality to report rent payments to any of the major cr...
25       Hello, Farhood. Thank you for reporting it. We'll fix this link.<a href="http://www.myassurantpo...
26       Anyway, I will;pass this issue to our QA to double check the functionality!;May this new year be...
28       Hello! I have checked tenant's account. The lease is shared and invoices are posted.But the TCPa...
29       I have reached out to Dwolla to speed up the verification. I'm waiting;for their response.I will...
30       So, if you

In [9]:
# response=df[df.parent_id != df.parent_id.isna()].message.fillna('0')
# q = df[df.parent_id.isna()].message.fillna('0')
# sub=df.subject

In [10]:
pd.set_option('display.max_colwidth', 100)

q = df[df.parent_id.isna()].message

question_df = pd.DataFrame(q)
question_df


In [2]:
!pip3 install simpleneighbors

    100% |████████████████████████████████| 645kB 3.2MB/s ta 0:00:011
  Running setup.py bdist_wheel for annoy ... done
  Stored in directory: /Users/sarahgibbons/Library/Caches/pip/wheels/9a/61/5c/2a83890a4531ed5177d6df7f667539316aa65a573cb4d44d71
Successfully built annoy


In [4]:
!python3 -m spacy download en_core_web_lg


    Linking successful
    /Users/sarahgibbons/anaconda3/lib/python3.7/site-packages/en_core_web_lg
    -->
    /Users/sarahgibbons/anaconda3/lib/python3.7/site-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [5]:
nlp = spacy.load('en_core_web_lg')

In [6]:
nlp.vocab['tenant'].vector # replace cheese with whatever word you want!

array([ 8.1999e-02,  2.6809e-01, -6.4207e-01, -2.0159e-01,  1.8809e-01,
       -5.3648e-01,  1.2361e-01, -9.8790e-01,  3.1961e-01,  2.2567e+00,
       -8.7102e-01, -7.2327e-02, -6.0627e-01,  5.6654e-01, -5.0096e-01,
        5.5449e-01,  5.9308e-01,  5.7918e-01,  6.6373e-02,  2.3008e-01,
       -1.1273e-01, -2.8318e-01,  1.5455e-01, -1.5975e-01,  3.2630e-01,
        4.0275e-02,  1.3673e-01,  1.0912e-01, -3.2834e-01, -1.8209e-01,
       -3.7159e-02, -9.1390e-02,  3.5177e-01,  5.1806e-01, -6.0034e-01,
        1.1090e-01, -8.3701e-02, -3.2436e-01, -1.7603e-01, -2.3717e-01,
       -2.1564e-01,  1.4902e-01,  2.4291e-01, -1.5415e-01, -4.2018e-01,
        9.4197e-02, -3.1186e-02, -3.0979e-02,  1.1423e-01,  6.5971e-01,
       -2.2602e-01,  1.7963e-01,  5.5509e-02, -2.2533e-01,  2.9006e-01,
       -3.5012e-01,  2.2379e-01, -8.3657e-02, -3.7734e-01,  5.1780e-01,
        4.8933e-01, -2.5230e-01, -8.3852e-02,  9.0077e-02,  3.6829e-01,
        1.9737e-02,  2.0779e-01,  3.0919e-01, -7.4070e-01,  4.15

In [60]:
feedback_string=question_df.message.to_string()
sentences = tokenize.sent_tokenize(feedback_string)
filtered_sent = [i for i in sentences if '?' in i]
filtered_sent[5]

"We all...\n124      I was trying to set up my account per my landlord's request and the system has locked me out and...\n125      Hi, how do I disable receiving an email notification when a TC message is received?"

In [64]:

def sentence_mean(nlp, s):
    s=str(filtered_sent)
    doc = nlp(s, disable=['tagger', 'parser'])
    return np.mean(np.array([w.vector for w in doc]), axis=0)
sentence_mean(nlp, "This... is a test.").shape

(300,)

In [65]:
from simpleneighbors import SimpleNeighbors

nns = SimpleNeighbors(300)
for i, line_id in enumerate(random.sample(list(responses.keys()), 10000)):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_mean(nlp, line_text)
    if np.any(summary_vector):
        nns.add_one(line_id, summary_vector)
nns.build()

NameError: name 'movie_lines' is not defined

In [87]:
sentence = "I like making bots."
picked = nns.nearest(sentence_mean(nlp, sentence), 4)[0:]
response_line_id = question_df[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", picked)



# print("Response to most similar turn:\n\t", df.message[response_line_id])

Your line:
	 I like making bots.
Most similar turn:
	 []


In [73]:
sentence = "I was trying to set up my account per my landlord\'s request and the system has locked me out"
picked = nns.nearest(nlp(sentence))
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
# print("Most similar turn:\n\t", filtered_sent[picked])
print("Response to most similar turn:\n\t", responses[response_line_id])

IndexError: Vector has wrong length (expected 300, got 20)

In [15]:
mystring =  feedback_string 
keyword = '?'
befor_keyowrd, keyword, after_keyword = mystring.partition(keyword)

In [16]:
after_keyword

' I...\n61                                                                                IIt wont accept my payment\n69       how do I add my own application to my listings. also can they download and fill it out or just f...\n74       Happy new year. I;<span style="font-family: inherit; font-size: inherit; font-style: inherit; fo...\n82       Some of our tenants are telling us they have to submit a photo ID for verification. And after 2 ...\n84       Hello, I\'ve been using Tenant Cloud for my residential rentals and love it. I am starting a new ...\n85       For rewards purposes most of my transactions are completed via CC. I would like a way to import ...\n103                 Are tenants able to split payments on their end?; Is it something I have to setup?Thanks\n107                                                  How do I see a copy of the email/invite to the tenents?\n110                   Getting a "request cannot be supported" response when I try to connect to Stacy Moles.\n1

In [10]:
tags_list = ['<p> ' ,' </p>' , '<*>', 
             '<b>', '</b>',' b>',
             '<ul>','</ul>','<li>',
             '</li>','<br>', '</br>', '↵',
             '-','<strong>','</strong>',
             '<span*>','</span>','<a href*>',
             '</a>', '<em> ','</em> ', '&', 'nbsp', 
             '&nbsp', 'â', 'µ','<div> ', ' </div>','/>', ';']


for tag in tags_list:
    question_df.message.replace(to_replace=tag, value = ' ', regex=True, inplace=True)

tokens = word_tokenize(question_df.message.to_string())  # tokenize with nltk 
tokens = [w.lower() for w in tokens] # convert to lower case


table = str.maketrans('', '', string.punctuation) # remove punctuation
stripped = [w.translate(table) for w in tokens]   

words = [word for word in stripped if word.isalpha()]

stop_words = set(stopwords.words('english')) # filter out stop words
words = [w for w in words if not w in stop_words]
words =[s for s in words if not s in tags_list]

texts = [words]
print(texts)

NameError: name 'stopwords' is not defined

In [ ]:
def get_top_n_words(text):
    text_dict = {}
    for line in text:
        line_spl = line.replace('.', ' ').replace('\x86','').replace('span',' ').replace('font', ' ').replace('hello', ' ').replace('?', ' ').replace('hi',' ').replace('(','').replace('div', '').split()
        for word in line_spl:
            lower_word = word.lower()
            if lower_word not in stop_words:
                if lower_word not in text_dict:
                    text_dict[lower_word] = 1
                else:
                    text_dict[lower_word] += 1
    max_dict = dict(sorted(text_dict.items(), key=operator.itemgetter(1), reverse=True)[:100]) 
    return max_dict

text_top = get_top_n_words(words)
print(text_top)

In [ ]:
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
model = Word2Vec(sentences = texts, size=100, window=5, min_count=5, workers=4, sg=0)

 _sentences_: list of split sentences 
* _size_: the dimensionality of the embedding vector
* _window_: the number of context words you are looking at
* _min_count_: tells the model to ignore words with total count less than this number.
* _workers_: the number of threads being used
* _sg_: whether to use skip-gram or CBOW (we will compare these two methods below

The __word embedding__ for the target words can obtained by extracting the hidden layers after feeding the __one-hot representation__ (which is a vector with only one target element being 1 and the others being 0) of that word into the network.

Essentially, we want to use the surrounding words to be the inputs to a Neural Network whose hidden layer encodes to the word.

With __skip-gram__, the representation dimension decreases from the vocabulary size (V) to the length of the hidden layer (N). Furthermore, the __vectors are more “meaningful” in terms of describing the relationship between words__. And we compose thse vectors by subtracting two related words sometimes express a meaningful concept such as gender or verb tense, as shown below.

One of the limitations of word2vec is is that it is not able to represent words that do not appear in the training dataset and even if we use a larger training set that contains more vocabulary there are some words that cant be mapped to vectors 

In [12]:
model.wv.most_similar("rent")

[('add', 0.9999276399612427),
 ('tenant', 0.9999196529388428),
 ('application', 0.9999144077301025),
 ('would', 0.999914288520813),
 ('hello', 0.9999141693115234),
 ('new', 0.9999122619628906),
 ('tenants', 0.999910831451416),
 ('way', 0.9999079704284668),
 ('credit', 0.9999075531959534),
 ('property', 0.9999061822891235)]